In [13]:
#importing the modules
import pandas as pd
import numpy as np
import matplotlib
import seaborn as sns
import plotly.express as px

#DATA COLLECTION

In [14]:
df= pd.read_csv("rent_apts.csv")
df.head(10)

,Agency,Neighborhood,Price,link,sq_mtrs,Bedrooms,Bathrooms
0,Buy Rent Shelters,"General Mathenge, Westlands","KSh 155,000",/listings/4-bedroom-apartment-for-rent-general...,4.0,4.0,4.0
1,Kenya Classic Homes,"Kilimani, Dagoretti North","KSh 100,000",/listings/3-bedroom-apartment-for-rent-kiliman...,300.0,3.0,4.0
2,Absolute Estate Agents,"Hatheru Rd,, Lavington, Dagoretti North","KSh 75,000",/listings/3-bedroom-apartment-for-rent-lavingt...,3.0,3.0,5.0
3,A1 Properties Limited,"Kilimani, Dagoretti North","KSh 135,000",/listings/3-bedroom-apartment-for-rent-kiliman...,227.0,3.0,4.0
4,Pmc Estates Limited,"Imara Daima, Embakasi","KSh 50,000",/listings/3-bedroom-apartment-for-rent-imara-d...,3.0,3.0,NaN
5,Lizmor Properties,"Waiyaki Way, Westlands","KSh 150,000",/listings/furnished-2-bedroom-apartment-for-re...,2.0,2.0,NaN
6,Mambo Properties Enterprise,"Off Denis Pritt, Kilimani, Dagoretti North","KSh 100,000",/listings/2-bedroom-apartment-for-rent-kiliman...,14.0,2.0,3.0
7,Wealthlink Realtors Ltd,"Westlands Area, Westlands","KSh 100,000",/listings/3-bedroom-apartment-for-rent-westlan...,186.0,3.0,3.0
8,Pmc Estates Limited,"Thika Road, Thika Road","KSh 85,000",/listings/4-bedroom-apartment-for-rent-thika-r...,4.0,4.0,NaN
9,Naftal Properties,"Kilimani, Dagoretti North","KSh 65,000",/listings/2-bedroom-apartment-for-rent-kiliman...,2.0,2.0,2.0


DATA CLEANING

In [20]:
df.shape


(1848, 7)

In [16]:

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1848 entries, 0 to 1847
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Agency        1848 non-null   object 
 1   Neighborhood  1848 non-null   object 
 2   Price         1848 non-null   object 
 3   link          1848 non-null   object 
 4   sq_mtrs       1846 non-null   float64
 5   Bedrooms      1845 non-null   float64
 6   Bathrooms     1557 non-null   float64
dtypes: float64(3), object(4)
memory usage: 101.2+ KB


In [21]:
df.isna().sum()  #Missing values

Agency            0
Neighborhood      0
Price             0
link              0
sq_mtrs           2
Bedrooms          3
Bathrooms       291
dtype: int64

There are missing values on the last three columns
These are float data types

In [22]:
#checking for number of unique values
df.nunique()

Agency           194
Neighborhood     647
Price            112
link            1761
sq_mtrs          154
Bedrooms           7
Bathrooms          6
dtype: int64

In [26]:
df.describe()

,sq_mtrs,Bedrooms,Bathrooms
count,1846.000000,1845.000000,1557.000000
mean,327.226977,2.577778,2.595376
std,2048.784581,0.819399,1.000349
min,0.000000,0.000000,1.000000
25%,2.000000,2.000000,2.000000
50%,3.000000,3.000000,2.000000
75%,4.000000,3.000000,3.000000
max,40470.000000,6.000000,6.000000


VISUALISATIONS
